<a href="https://colab.research.google.com/github/syunar/22-01_Laptops-Prices-Prediction-Analysis/blob/main/22-01-01_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
# install selenium and chrimedriver
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.1 MB/s 
     |████████████████████████████████| 358 kB 37.6 MB/s 
     |████████████████████████████████| 139 kB 54.2 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 4.1 MB 42.5 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://

# Import Libraries

In [2]:
# import libraries
from selenium.webdriver.common.by import By
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
import pandas as pd
import time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


# 8 Steps for Web Scraping:
1. get into first page.
2. find the number and elements in webstructure of each products on 1 page.
3. collect the data like name, price, url of each products by pointer element from 2.
4. loop to page 2, 3, ... , n.
5. get into each product_urls.
6. find element of data e.g. CPU, GPU, model_year, size, refresh_rate, etc.
7. collect data to dict (1 laptop per 1 dict)
8. loop each product_urls -> collect to list of dict(JSON)


In [3]:
# n_product = 21 
# pages = int(n_product / 21) 
pages = 2
all_products = [] 

# loop to each pages to acquire datas
for page in range(1,pages+1):
  
  url = f'https://notebookspec.com/notebook/search?page={page}'
  driver.get(url)
  
  n_products = len(driver.find_elements(By.CSS_SELECTOR, '.item-info .title'))
  products = range(1,n_products+1)

  # get into each product div and collect data only inside the div
  for product in products:

    get_names = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) .title')
    get_prices = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) span')
    get_urls = driver.find_elements(By.CSS_SELECTOR, f'.search-result-item:nth-child({product}) .item-info a')
    
    names = [name.get_attribute('innerText').strip() for name in get_names]
    prices = [price.get_attribute('innerText').strip() for price in get_prices]
    urls = [url.get_attribute('href').strip() for url in get_urls]

    print(f'get urls: page={page}, product_count={product}, {names[0]}')

    keys = ['names', 'prices', 'urls'] # combine the keys
    values = names + prices + urls # combine the values

    # convert list of keys and values to dict
    tempdata_dict = {}
    tempdata_dict.update({k:v for k,v in zip(keys,values)})
    all_products.append(tempdata_dict)
  # end for loop products on each page
# end for loop pages


# loop to each products link url to collect more data
  count = 1
for product in all_products:
  url = product['urls']
  driver.get(url)

  get_keys = driver.find_elements(By.CSS_SELECTOR, '.specificate-title')
  get_values = driver.find_elements(By.CSS_SELECTOR, '.specificate-info')
  keys = [key.get_attribute('innerText') for key in get_keys]
  values = [value.get_attribute('innerText') for value in get_values]

  product.update({k:v for k,v in zip(keys,values)})

  print(f'get infos: product_count={count}, {product["names"]}')
  count += 1
# end for loop product_urls

df = pd.DataFrame(all_products)
print(df.info())





get urls: page=1, product_count=1, Huawei MateBook X Pro-i7/16/1TB
get urls: page=1, product_count=2, Huawei MateBook D 16-i7/16/512
get urls: page=1, product_count=3, Huawei MateBook D 16-i5/16/512
get urls: page=1, product_count=4, Asus ZenBook S 13 OLED UM5302TA-LV511WS
get urls: page=1, product_count=5, Lenovo Legion 5 15ARH7H-82RD0041TA
get urls: page=1, product_count=6, Lenovo Legion 5 15ARH7H-82RB00A1TA
get urls: page=1, product_count=7, Lenovo IdeaPad Gaming 3 15IAH7-82S900JHTA
get urls: page=1, product_count=8, Acer Aspire 3 A315-58-774U
get urls: page=1, product_count=9, HP ProBook 245 G8-8F7TU
get urls: page=1, product_count=10, Lenovo IdeaPad L3 15ITL6-82HL00GETA
get urls: page=1, product_count=11, Lenovo IdeaPad L3 15ITL6-82HL00GFTA
get urls: page=1, product_count=12, Lenovo IdeaPad Gaming 3 15IHU6-82K1019KTA
get urls: page=1, product_count=13, Lenovo IdeaPad Gaming 3 15IAH7-82S9007GTA
get urls: page=1, product_count=14, Lenovo IdeaPad Gaming 3 15IAH7-82S900JGTA
get urls: 

In [4]:
df.head()

,names,prices,urls,Brand,Model,Model Year,CPU,GPU,Panel Type,Size,...,Body Color,Weight,Battery Type,Keyboard Type,Keyboard Backlit,Numpad,Other Detail,Warranty,Insurance,
0,Huawei MateBook X Pro-i7/16/1TB,"79,990",https://notebookspec.com/notebook/12172-huawei...,Huawei,Huawei MateBook X Pro-i7/16/1TB,Q3 / 2022,"Intel Core i7-1260P (1.50 GHz, 18 MB L3 Cache ...",Intel Iris Xe Graphics G7,IPS,14.2 นิ้ว,...,Ink Blue,1.26 kg,"Li-Polymer, 60Wh",Chiclet Keyboard,มี,ไม่มี,Microsoft Office Home & Student 2021,2 Y. Carry-in Regional,ไม่มี,NaN
1,Huawei MateBook D 16-i7/16/512,"39,990",https://notebookspec.com/notebook/12171-huawei...,Huawei,Huawei MateBook D 16-i7/16/512,Q3 / 2022,"Intel Core i7-12700H (3.50 GHz, 24 MB L3 Cache...",Intel Iris Xe Graphics G7,IPS,16 นิ้ว,...,Mystic Silver,1.7 kg,"Li-Polymer, 60Wh",Chiclet Keyboard,มี,มี,Microsoft Office Home & Student 2021,2 Y. Carry-in Regional,ไม่มี,2x USB 3.2 Type C
2,Huawei MateBook D 16-i5/16/512,"33,990",https://notebookspec.com/notebook/12170-huawei...,Huawei,Huawei MateBook D 16-i5/16/512,Q3 / 2022,"Intel Core i5-12450H (3.30 GHz, 12 MB L3 Cache...",Intel UHD Graphics Xe G4,IPS,16 นิ้ว,...,Mystic Silver,1.7 kg,"Li-Polymer, 60Wh",Chiclet Keyboard,มี,มี,Microsoft Office Home & Student 2021,2 Y. Carry-in Regional,ไม่มี,2x USB 3.2 Type C
3,Asus ZenBook S 13 OLED UM5302TA-LV511WS,"31,990",https://notebookspec.com/notebook/12169-asus-z...,Asus,Asus ZenBook S 13 OLED UM5302TA-LV511WS,Q3 / 2022,"AMD Ryzen 5 6600U (2.90 GHz up to 4.50 GHz, 16...",AMD Radeon RX Vega 6,OLED,13.3 นิ้ว,...,Ponder Blue,1 kg,"4-Cell Li-ion, 67Whr",Chiclet Keyboard,มี,มี,Microsoft Office Home & Student 2021,2Y Onsite + 1Y Perfect,ไม่มี,NaN
4,Lenovo Legion 5 15ARH7H-82RD0041TA,"67,990",https://notebookspec.com/notebook/12168-lenovo...,Lenovo,Lenovo Legion 5 15ARH7H-82RD0041TA,Q3 / 2022,"AMD Ryzen 7 6800H (3.20 GHz up to 4.70 GHz, 16...",NVIDIA GeForce RTX 3070 (8GB GDDR6),IPS,15.6 นิ้ว,...,Storm Grey,2.4 kg,"4-Cell Li-Polymer, 80Wh",RGB Chiclet Keyboard,มี,มี,Office Trial,3Y Onsite + 2Y ADP,ไม่มี,3x USB 3.2 Type C


# Export to CSV

In [5]:
df.to_csv('nbspec.csv')